# Using our Enviroment

Before using our custom enviroment we must use pip to install it.

In [ ]:
!pip install --user --upgrade --editable Assignment-1

It's better to restart the kernel once the pip is done installing.

Initializing all the required parameters in a dictionary named params

Set global seed for both random and numpy.random

In [ ]:
import gymnasium as gym
import numpy as np
import math
import random
from matplotlib import pyplot as plt
random.seed(2023)
np.random.seed(2023)
params={}
params["sigma"]=1
params["mu"]=0
params["arms"]=10
params["maxEpisodes"]=1000
params["epsilon"]=0.1
params["initial_epsilon"]=1.0
params["final_epsilon"]={}
params["decay_rate"]={}
params["final_epsilon"]['lin']=0.0
params["final_epsilon"]['exp']=0.0001
params["decay_rate"]['lin']=(params["initial_epsilon"]-params["final_epsilon"]['lin'])/params["maxEpisodes"]
params["decay_rate"]['exp']=np.log(params["initial_epsilon"]/params["final_epsilon"]['exp'])/params["maxEpisodes"]
params["c_UCB"]=0.1
params["init_temp"]=10000000
params["final_temp"]=0.05
params["temp_decay_rate_lin"]=(params["init_temp"]-params["final_temp"])/params["maxEpisodes"]
params["temp_decay_rate_exp"]=np.log(params["init_temp"]/params["final_temp"])/params["maxEpisodes"]
params["no_envs"]=50



In [ ]:
# We now simply Import gym, create our environment and run a few steps.

In [ ]:
random.seed(2023)
np.random.seed(2023)
env=gym.make('Assignment_1:tenArmedGaussianBandit-v0',mu=params["mu"],sigma=params["sigma"],arms=params["arms"])
#env=gym.make('Assignment_1:tenArmedGaussianBandit-v0',mu=0.0,sigma=1.0,arms=params["arms"])
episodes = 10
for episode in range(1,episodes+1):
    state=env.reset(seed=2023)
    done= False
    action =random.choice(list(range(0,env.action_space.n)))
    end_state, reward, done, truncated, info =env.step(action)
    
    print('Episode:{} End State: {} Reward:{} Action:{}'.format(episode,end_state,reward,action))

Declaring Initial helper function Softmax

In [ ]:
def softMax(x):
    return np.exp(x)/(np.sum(np.exp(x)))

Pure Exploitation

In [ ]:
def PureExploitation(env,params):
    # print("PureExploitation")
    Q = np.zeros(env.action_space.n)
    N = np.zeros(env.action_space.n)
    e = 0
    Q_est = np.zeros((params["maxEpisodes"],env.action_space.n))
    R=np.zeros((params["maxEpisodes"]))
    actions=np.zeros((params["maxEpisodes"]))
    env.reset(seed=2023)
    while e < params["maxEpisodes"]-1 :
        max_indices=np.where(Q==np.amax(Q))
        action = random.choice(max_indices[0])
        # print(action)
        # action = np.argmax(Q)
        end_state, reward, done, truncated, info = env.step(action)
        N[action] = N[action] + 1
        Q[action] = Q[action] + (reward-Q[action])/N[action]
        e = e+1
        Q_est[e] = Q
        R[e]=reward
        actions[e]=action
        env.reset(seed=2023)
    return Q_est,R,actions
# print(PureExploitation(env,params))

Pure Exploration

In [ ]:
def PureExploration(env,params):
    # print("PureExploration")
    Q = np.zeros(env.action_space.n)
    N = np.zeros(env.action_space.n)
    e = 0
    Q_est = np.zeros((params["maxEpisodes"],env.action_space.n))
    R=np.zeros((params["maxEpisodes"]))
    actions=np.zeros((params["maxEpisodes"]))
    env.reset(seed=2023)
    while e < params["maxEpisodes"]-1 :
        action= random.choice((0,len(Q)-1))
        # print(action)
        end_state, reward, done,truncated,info = env.step(action)
        N[action] = N[action] + 1
        Q[action] = Q[action] + (reward-Q[action])/N[action]
        e = e+1
        Q_est[e] = Q
        R[e]=reward
        actions[e]=action
        env.reset(seed=2023)
    return Q_est,R,actions

Epsilon Greedy 

In [ ]:
def epsilonGreedy(env,params):
    # print("epsilonGreedy")
    Q = np.zeros(env.action_space.n)
    N = np.zeros(env.action_space.n)
    e = 0
    Q_est = np.zeros((params["maxEpisodes"],env.action_space.n))
    R=np.zeros((params["maxEpisodes"]))
    actions=np.zeros((params["maxEpisodes"]))
    env.reset(seed=2023)
    while e < params["maxEpisodes"]-1 :
        if random.random() > params["epsilon"]:
            max_indices=np.where(Q==np.amax(Q))
            action = random.choice(max_indices[0])
        else :
            action= random.choice((0,len(Q)-1))
        # print(action)
        end_state, reward, done,truncated,info = env.step(action)
        N[action] = N[action] + 1
        Q[action] = Q[action] + (reward-Q[action])/N[action]
        e = e+1
        Q_est[e] = Q
        R[e]=reward
        actions[e]=action
        env.reset(seed=2023)
    return Q_est,R,actions

decaying Epsilon Greedy

In [ ]:
def decayingEpsilonGreedy(env,params,type):
    # print("decayingEpsilonGreedy")
    Q = np.zeros(env.action_space.n)
    N = np.zeros(env.action_space.n)
    e = 0
    Q_est = np.zeros((params["maxEpisodes"],env.action_space.n))
    R=np.zeros((params["maxEpisodes"]))
    actions=np.zeros((params["maxEpisodes"]))
    env.reset(seed=2023)
    epsilon=params["initial_epsilon"]
    while e < params["maxEpisodes"]-1 :
        if random.random() > epsilon:
            max_indices=np.where(Q==np.amax(Q))
            action = random.choice(max_indices[0])
        else :
            action= random.choice((0,len(Q)-1))
        # print(action)
        end_state, reward, done, truncated, info = env.step(action)
        N[action] = N[action] + 1
        Q[action] = Q[action] + (reward-Q[action])/N[action]
        e = e+1
        Q_est[e] = Q
        
        if type=='lin':
            epsilon=epsilon-params["decay_rate"]['lin']
        else :
            epsilon = epsilon*np.exp(-params["decay_rate"]['exp'])
            
        R[e]=reward
        actions[e]=action
        env.reset(seed=2023)
    return Q_est,R,actions


SoftMax Exploration

In [ ]:
def softMaxExploration(env,params,type):
    # print("softMaxExploration")
    Q = np.zeros(env.action_space.n)
    N = np.zeros(env.action_space.n)
    e = 0
    Q_est = np.zeros((params["maxEpisodes"],env.action_space.n))
    R=np.zeros((params["maxEpisodes"]))
    actions=np.zeros((params["maxEpisodes"]))
    env.reset(seed=2023)
    temp=params["init_temp"]
    while e < params["maxEpisodes"]-1 :
        probs = softMax(Q/temp)
        action=random.choices(list(range(0,env.action_space.n)),weights=probs,k=1)[0]
        # print(action)
        end_state, reward, done, truncated, info = env.step(action)
        N[action] = N[action] + 1
        Q[action] = Q[action] + (reward-Q[action])/N[action]
        # have to decay temperature
        e = e+1
        Q_est[e] = Q
        if type=='lin':
            temp=temp-params["temp_decay_rate_lin"]
        else :
            temp = temp*np.exp(-params["temp_decay_rate_exp"])
        R[e]=reward
        actions[e]=action
        env.reset(seed=2023)
    return Q_est,R,actions

Upper Confidence Exploration

In [ ]:
def UCBexploration(env,params):
    # print("UCBexploration")
    Q = np.zeros(env.action_space.n)
    N = np.zeros(env.action_space.n)

    e = 0
    Q_est = np.zeros((params["maxEpisodes"],env.action_space.n))
    R=np.zeros((params["maxEpisodes"]))
    actions=np.zeros((params["maxEpisodes"]))
    env.reset(seed=2023)
    while e < params["maxEpisodes"] - 1:
        if e< len(Q):
            action = e
        else:
            U= params["c_UCB"]* math.sqrt(math.log(e))/np.sqrt(N)
            UCB = np.add(Q,U)
            max_indices=np.where(UCB==np.amax(UCB))
            action = random.choice(max_indices[0])
        end_state, reward, done, truncated, info = env.step(action)
        N[action] = N[action] + 1
        Q[action] = Q[action] + (reward-Q[action])/N[action]
        e = e+1
        Q_est[e] = Q
        R[e] =reward
        actions[e]=action
        env.reset(seed=2023)
    return Q_est,R,actions

Creating 50 different probability distribution for creating 50 different Environments. Also simultaneously creating V* and Q* tables.

In [ ]:
random.seed(2023)
np.random.seed(2023)
envs=[]
q_star=[]
v_star=[]
for i in range(50):
        # envs is a list of environments
    env=gym.make('Assignment_1:tenArmedGaussianBandit-v0',mu=params["mu"],sigma=params["sigma"],arms=params["arms"])
    envs.append(env)
    q_star.append(envs[i].mean_rewards)
    v_star.append(np.amax(q_star[i]))

Average Reward Plot for different Agents with timesteps, Initializing decay 

In [ ]:
decay_type="exp"

In [ ]:
random.seed(2023)
np.random.seed(2023)
Rewards1=[]
for i in range(params["no_envs"]):
        
    Q_est,R,actions =PureExploitation(envs[i],params)
    # print(np.shape(R))
    Rewards1.append(R)
    # print(i)
Rewards1=np.transpose(Rewards1)    
# print(Rewards[23])
avg_rewards1=[]
for i in range(params["maxEpisodes"]):
    avg_rewards1.append(np.sum(Rewards1[i])/params["no_envs"])
# print(np.shape(avg_rewards))
plt.plot(np.arange(0,params["maxEpisodes"]),avg_rewards1, color='r')
# plt.ylabel('Average Rewards per timesteps')  
# plt.xlabel('Timesteps') 
# plt.title('PureExploitation:Average rewards recieved vs timestep 10armedBandit') 
# plt.savefig(' PureExploitation-10armedBandit Average rewards recieved vs timestep .pdf') 
# # plt.close()
# # print(np.shape(Rewards1))

Rewards2=[]
for i in range(params["no_envs"]):
        
    Q_est,R ,actions=PureExploration(envs[i],params)

    Rewards2.append(R)
    # print(i)
Rewards2=np.transpose(Rewards2)    
    
avg_rewards2=[]
for i in range(params["maxEpisodes"]):
    avg_rewards2.append(np.sum(Rewards2[i])/params["no_envs"])
# print(np.shape(avg_rewards))
plt.plot(np.arange(0,params["maxEpisodes"]),avg_rewards2, color='blue')
# plt.ylabel('Average Rewards per timesteps')  
# plt.xlabel('Timesteps')  
# plt.title('PureExploration:Average rewards recieved vs timestep 10armedBandit') 
# plt.savefig(' PureExploration-10armedBandit Average rewards recieved vs timestep .pdf') 
# # plt.close()

Rewards3=[]
for i in range(params["no_envs"]):
    
    Q_est,R ,actions=epsilonGreedy(envs[i],params)
    # print(np.shape(R))
    Rewards3.append(R)
    # print(i)
Rewards3=np.transpose(Rewards3)    
# print(Rewards[23])
avg_rewards3=[]
for i in range(params["maxEpisodes"]):
    avg_rewards3.append(np.sum(Rewards3[i])/params["no_envs"])
# print(np.shape(avg_rewards))
plt.plot(np.arange(0,params["maxEpisodes"]),avg_rewards3, color='green')
# plt.ylabel('Average Rewards per timesteps')  
# plt.xlabel('Timesteps')  
# plt.title('epsilonGreedy:Average rewards recieved vs timestep 10armedBandit') 
# plt.savefig(' epsilonGreedy- 10armedBandit Average rewards recieved vs timestep.pdf') 
# # plt.close()

Rewards4=[]
for i in range(params["no_envs"]):
    
    Q_est,R ,actions=decayingEpsilonGreedy(envs[i],params,decay_type)
    # print(np.shape(R))
    Rewards4.append(R)
    # print(i)
Rewards4=np.transpose(Rewards4)    
# print(Rewards[23])
avg_rewards4=[]
for i in range(params["maxEpisodes"]):
    avg_rewards4.append(np.sum(Rewards4[i])/params["no_envs"])
# print(np.shape(avg_rewards))
plt.plot(np.arange(0,params["maxEpisodes"]),avg_rewards4, color='black')
#plt.ylabel('Average Rewards per timesteps')  
#plt.xlabel('Timesteps')  
#plt.title('decayingEpsilonGreedy:Average rewards recieved vs timestep 10armedBandit') 
#plt.savefig(' decayingEpsilonGreedy-10armedBandit Average rewards recieved vs timestep .pdf') 
# # plt.close()

Rewards5=[]
for i in range(params["no_envs"]):
    
    Q_est,R ,actions=softMaxExploration(envs[i],params,decay_type)
    # print(np.shape(R))
    Rewards5.append(R)
    # print(i)
Rewards5=np.transpose(Rewards5)    
# print(Rewards[23])
avg_rewards5=[]
for i in range(params["maxEpisodes"]):
    avg_rewards5.append(np.sum(Rewards5[i])/params["no_envs"])
# print(np.shape(avg_rewards))
plt.plot(np.arange(0,params["maxEpisodes"]),avg_rewards5,color='pink')
# plt.ylabel('Average Rewards per timesteps')  
# plt.xlabel('Timesteps')  
# plt.title('softMaxExploration:Average rewards recieved vs timestep 10armedBandit') 
# plt.savefig('softMaxExploration-10armedBandit Average rewards recieved vs timestep .pdf') 
# # plt.close()

Rewards6=[]
for i in range(params["no_envs"]):
    
    Q_est,R,actions =UCBexploration(envs[i],params)
    # print(np.shape(R))
    Rewards6.append(R)
    # print(i)
Rewards6=np.transpose(Rewards6)    
# print(Rewards[23])
avg_rewards6=[]
for i in range(params["maxEpisodes"]):
    avg_rewards6.append(np.sum(Rewards6[i])/params["no_envs"])
# print(np.shape(avg_rewards))
#plt.legend(['Pure Exploitation', 'Pure Exploration', 'Epsilon Greedy', 'Decaying Epsilon Greedy', 'SoftMax Exploration', 'UCB Exploration'], loc='best')
plt.plot(np.arange(0,params["maxEpisodes"]),avg_rewards6, color='orange')
# plt.ylabel('Average Rewards per timesteps')  
# plt.xlabel('Timesteps')  
# plt.title('UCBexploration:Average rewards recieved vs timestep 10armedBandit') 
# plt.savefig(' UCBexploration-10armedBandit Average rewards recieved vs timestep .pdf') 
plt.title("Average Rewards of Different Agents: 10 armed")
plt.xlabel('Timesteps')
plt.ylabel("Average Rewards")
plt.savefig("Average Rewards of Different Agents-10 armed bandit.pdf")
# plt.close()   

Average Regret for different Agents

In [ ]:
random.seed(2023)
np.random.seed(2023)
Actions1=np.zeros((params["no_envs"],params["maxEpisodes"]))
Q_est1=np.empty((params["no_envs"],envs[0].action_space.n))
Regret1=np.zeros(params["maxEpisodes"])
for i in range(params["no_envs"]):
    
    Q_est,R,actions=PureExploitation(envs[i],params)
    # print(actions)
    # print(np.shape(actions))
    Actions1[i]=actions

Actions1=np.transpose(Actions1)
for i in range(params["maxEpisodes"]):
    if i==0:
        sum = 0
        for j in range(params["no_envs"]):
            sum+=(v_star[j]-q_star[j][int(Actions1[i][j])])
            # print(sum)
        # print(np.shape(sum))
        Regret1[i]=sum/params["no_envs"]
        # Regret1=np.average(np.amax(Q_est1,axis=1)-Q_est[Actions1[i]])
    else:
        sum =0
        for j in range(params["no_envs"]):
            sum+=(v_star[j]-q_star[j][int(Actions1[i][j])])
        Regret1[i]=Regret1[i-1]+(sum/params["no_envs"])
# print(Regret1)
plt.plot(np.arange(0,params["maxEpisodes"]),Regret1, color='r')
# plt.ylabel('Average Regrets per timesteps')  
# plt.xlabel('Timesteps') 
# plt.title('PureExploitation:Average regrets recieved vs timestep 2armedBandit') 
# plt.savefig(' PureExploitation-2armedBandit Average regrets recieved vs timestep .pdf') 
# plt.close()

Actions2=np.zeros((params["no_envs"],params["maxEpisodes"]))
Q_est2=np.empty((params["no_envs"],envs[0].action_space.n))
Regret2=np.zeros(params["maxEpisodes"])
for i in range(params["no_envs"]):
    
    Q_est,R,actions=PureExploration(envs[i],params)
    # print(actions)
    # print(np.shape(actions))
    Actions2[i]=actions
    # print(Q_est[-1])
    # Q_est2[i]=Q_est[-1]
# print(np.shape(Actions1))
# print(np.shape(Q_est1))
Actions2=np.transpose(Actions2)
for i in range(params["maxEpisodes"]):
    if i==0:
        sum = 0
        for j in range(params["no_envs"]):
            sum+=(v_star[j]-q_star[j][int(Actions2[i][j])])
            # print(sum)
        # print(np.shape(sum))
        Regret2[i]=sum/params["no_envs"]
        # Regret1=np.average(np.amax(Q_est1,axis=1)-Q_est[Actions1[i]])
    else:
        sum =0
        for j in range(params["no_envs"]):
            sum+=(v_star[j]-q_star[j][int(Actions2[i][j])])
        Regret2[i]=Regret2[i-1]+(sum/params["no_envs"])
# print(Regret1)
plt.plot(np.arange(0,params["maxEpisodes"]),Regret2, color='blue')
# plt.ylabel('Average Regrets per timesteps')  
# plt.xlabel('Timesteps') 
# plt.title('PureExploration:Average regrets recieved vs timestep 2armedBandit') 
# plt.savefig(' PureExploration-2armedBandit Average regrets recieved vs timestep .pdf') 
# plt.close()

Actions1=np.zeros((params["no_envs"],params["maxEpisodes"]))
Q_est1=np.empty((params["no_envs"],envs[0].action_space.n))
Regret1=np.zeros(params["maxEpisodes"])
for i in range(params["no_envs"]):
    
    Q_est,R,actions=epsilonGreedy(envs[i],params)
    # print(actions)
    # print(np.shape(actions))
    Actions1[i]=actions
    # print(Q_est[-1])
    # Q_est1[i]=Q_est[-1]
# print(np.shape(Actions1))
# print(np.shape(Q_est1))
Actions1=np.transpose(Actions1)
for i in range(params["maxEpisodes"]):
    if i==0:
        sum = 0
        for j in range(params["no_envs"]):
            sum+=(v_star[j]-q_star[j][int(Actions1[i][j])])
            # print(sum)
        # print(np.shape(sum))
        Regret1[i]=sum/params["no_envs"]
        # Regret1=np.average(np.amax(Q_est1,axis=1)-Q_est[Actions1[i]])
    else:
        sum =0
        for j in range(params["no_envs"]):
            sum+=(v_star[j]-q_star[j][int(Actions1[i][j])])
        Regret1[i]=Regret1[i-1]+sum/params["no_envs"]
# print(Regret1)
plt.plot(np.arange(0,params["maxEpisodes"]),Regret1, color='green')
# plt.ylabel('Average Regrets per timesteps')  
# plt.xlabel('Timesteps') 
# plt.title('epsilonGreedy:Average regrets recieved vs timestep 2armedBandit') 
# plt.savefig(' epsilonGreedy-2armedBandit Average regrets recieved vs timestep .pdf') 
# plt.close()

Actions1=np.zeros((params["no_envs"],params["maxEpisodes"]))
Q_est1=np.empty((params["no_envs"],envs[0].action_space.n))
Regret1=np.zeros(params["maxEpisodes"])
for i in range(params["no_envs"]):
        
    Q_est,R,actions=decayingEpsilonGreedy(envs[i],params,decay_type)
    # print(actions)
    # print(np.shape(actions))
    Actions1[i]=actions
    # print(Q_est[-1])
    # Q_est1[i]=Q_est[-1]
# print(np.shape(Actions1))
# print(np.shape(Q_est1))
Actions1=np.transpose(Actions1)
for i in range(params["maxEpisodes"]):
    if i==0:
        sum = 0
        for j in range(params["no_envs"]):
            sum+=(v_star[j]-q_star[j][int(Actions1[i][j])])
            # print(sum)
        # print(np.shape(sum))
        Regret1[i]=sum/params["no_envs"]
        # Regret1=np.average(np.amax(Q_est1,axis=1)-Q_est[Actions1[i]])
    else:
        sum =0
        for j in range(params["no_envs"]):
            sum+=(v_star[j]-q_star[j][int(Actions1[i][j])])
        Regret1[i]=Regret1[i-1]+sum/params["no_envs"]
    # print(Regret1)
plt.plot(np.arange(0,params["maxEpisodes"]),Regret1, color='black')
# plt.ylabel('Average Regrets per timesteps')  
# plt.xlabel('Timesteps') 
# plt.title('decayingEpsilonGreedy:Average regrets recieved vs timestep 2armedBandit') 
# plt.savefig(' decayingEpsilonGreedy-2armedBandit Average regrets recieved vs timestep .pdf') 
# plt.close()

Actions1=np.zeros((params["no_envs"],params["maxEpisodes"]))
Q_est1=np.empty((params["no_envs"],envs[0].action_space.n))
Regret1=np.zeros(params["maxEpisodes"])
for i in range(params["no_envs"]):

    Q_est,R,actions=softMaxExploration(envs[i],params,decay_type)
    # print(actions)
    # print(np.shape(actions))
    Actions1[i]=actions
    # print(Q_est[-1])
    # Q_est1[i]=Q_est[-1]
# print(np.shape(Actions1))
# print(np.shape(Q_est1))
Actions1=np.transpose(Actions1)
for i in range(params["maxEpisodes"]):
    if i==0:
        sum = 0
        for j in range(params["no_envs"]):
            sum+=(v_star[j]-q_star[j][int(Actions1[i][j])])
            # print(sum)
        # print(np.shape(sum))
        Regret1[i]=sum/params["no_envs"]
    # Regret1=np.average(np.amax(Q_est1,axis=1)-Q_est[Actions1[i]])
    else:
        sum =0
        for j in range(params["no_envs"]):
            sum+=(v_star[j]-q_star[j][int(Actions1[i][j])])
        Regret1[i]=Regret1[i-1]+sum/params["no_envs"]
# # print(Regret1)
plt.plot(np.arange(0,params["maxEpisodes"]),Regret1, color='pink')
# plt.ylabel('Average Regrets per timesteps')  
# plt.xlabel('Timesteps') 
# plt.title('softMaxExploration:Average regrets recieved vs timestep 2armedBandit') 
# plt.savefig(' softMaxExploration-2armedBandit Average regrets recieved vs timestep .pdf') 
# plt.close()

Actions1=np.zeros((params["no_envs"],params["maxEpisodes"]))
Q_est1=np.empty((params["no_envs"],envs[0].action_space.n))
Regret1=np.zeros(params["maxEpisodes"])
for i in range(params["no_envs"]):
    
    Q_est,R,actions=UCBexploration(envs[i],params)
    # print(actions)
    # print(np.shape(actions))
    Actions1[i]=actions
    # print(Q_est[-1])
    # Q_est1[i]=Q_est[-1]
# print(np.shape(Actions1))
# print(np.shape(Q_est1))
Actions1=np.transpose(Actions1)
for i in range(params["maxEpisodes"]):
    if i==0:
        sum = 0
        for j in range(params["no_envs"]):
            sum+=(v_star[j]-q_star[j][int(Actions1[i][j])])
            # print(sum)
        # print(np.shape(sum))
        Regret1[i]=sum/params["no_envs"]
            # Regret1=np.average(np.amax(Q_est1,axis=1)-Q_est[Actions1[i]])
    else:
        sum =0
        for j in range(params["no_envs"]):
                sum+=(v_star[j]-q_star[j][int(Actions1[i][j])])
        Regret1[i]=Regret1[i-1]+sum/params["no_envs"]
# print(Regret1)
plt.plot(np.arange(0,params["maxEpisodes"]),Regret1, color='orange')
plt.ylabel('Average Regrets per timesteps')  
plt.xlabel('Timesteps') 
# plt.title('UCBexploration:Average regrets recieved vs timestep 2armedBandit') 
plt.legend(["Pure Exploitation","Pure Exploration","Epsilon-Greedy","Decaying Epsilon Greedy","Softmax exploration","UCB Exploration"], loc ="upper left")
plt.title("Regret for different Agents: 10 armed")
plt.savefig('Regrets 10 armed.pdf') 
# plt.index
# plt.close()
# Regret(envs,params,v_star,q_star,"exp")

% of Optimal Actions taken by different Agents

In [ ]:
random.seed(2023)
np.random.seed(2023)
Actions1=np.zeros((params["no_envs"],params["maxEpisodes"]))
Q_est1=np.zeros((params["no_envs"],envs[0].action_space.n))
for i in range(params["no_envs"]):
    Q_est,R,actions=PureExploitation(envs[i],params)
    # print(actions)
    # print(np.shape(actions))
    Actions1[i]=actions
    # print(Q_est[-1])
    # Q_est1[i]=Q_est[-1]
Actions1=np.transpose(Actions1)
# print(np.shape(Actions1))
optimal1=np.zeros((params["maxEpisodes"]))     
for i in range(params["maxEpisodes"]):
    count1=0
    for j in range(params["no_envs"]):
        if Actions1[i][j] in np.where(q_star[j]==v_star[j]):
            count1+=1
    optimal1[i]=(count1*100/params["no_envs"])
plt.plot(np.arange(0,params["maxEpisodes"]),optimal1, color='r')
# plt.ylabel('Average Regrets per timesteps')  
# plt.xlabel('Timesteps') 
# plt.title('PureExploitation: % optimal action done vs timestep 2armedBandit') 
# plt.savefig(' PureExploitation-2armedBandit  % optimal action done  vs timestep .pdf') 
# plt.close()

Actions1=np.zeros((params["no_envs"],params["maxEpisodes"]))
Q_est1=np.zeros((params["no_envs"],envs[0].action_space.n))
for i in range(params["no_envs"]):
    Q_est,R,actions=PureExploration(envs[i],params)
    # print(actions)
    # print(np.shape(actions))
    Actions1[i]=actions
    # print(Q_est[-1])
    # Q_est1[i]=Q_est[-1]
Actions1=np.transpose(Actions1)
# print(np.shape(Actions1))
optimal1=np.zeros((params["maxEpisodes"]))     
for i in range(params["maxEpisodes"]):
    count1=0
    for j in range(params["no_envs"]):
        if Actions1[i][j] in np.where(q_star[j]==v_star[j]):
            count1+=1
    optimal1[i]=(count1*100/params["no_envs"])
plt.plot(np.arange(0,params["maxEpisodes"]),optimal1, color='blue')
# plt.ylabel('Average Regrets per timesteps')  
# plt.xlabel('Timesteps') 
# plt.title('PureExploration: % optimal action done vs timestep 2armedBandit') 
# plt.savefig('PureExploration-2armedBandit  % optimal action done  vs timestep .pdf') 
# plt.close()

Actions1=np.zeros((params["no_envs"],params["maxEpisodes"]))
Q_est1=np.zeros((params["no_envs"],envs[0].action_space.n))
for i in range(params["no_envs"]):
    Q_est,R,actions=epsilonGreedy(envs[i],params)
    # print(actions)
    # print(np.shape(actions))
    Actions1[i]=actions
    # print(Q_est[-1])
    # Q_est1[i]=Q_est[-1]
Actions1=np.transpose(Actions1)
# print(np.shape(Actions1))
optimal1=np.zeros((params["maxEpisodes"]))     
for i in range(params["maxEpisodes"]):
    count1=0
    for j in range(params["no_envs"]):
        if Actions1[i][j] in np.where(q_star[j]==v_star[j]):
            count1+=1
    optimal1[i]=(count1*100/params["no_envs"])
plt.plot(np.arange(0,params["maxEpisodes"]),optimal1, color='green')
# plt.ylabel('Average Regrets per timesteps')  
# plt.xlabel('Timesteps') 
# plt.title('epsilonGreedy: % optimal action done vs timestep 2armedBandit') 
# plt.savefig('epsilonGreedy-2armedBandit  % optimal action done  vs timestep .pdf') 
# plt.close()

Actions1=np.zeros((params["no_envs"],params["maxEpisodes"]))
Q_est1=np.zeros((params["no_envs"],envs[0].action_space.n))
for i in range(params["no_envs"]):
    Q_est,R,actions=decayingEpsilonGreedy(envs[i],params,decay_type)
    # print(actions)
    # print(np.shape(actions))
    Actions1[i]=actions
    # print(Q_est[-1])
    # Q_est1[i]=Q_est[-1]
Actions1=np.transpose(Actions1)
# print(np.shape(Actions1))
optimal1=np.zeros((params["maxEpisodes"]))     
for i in range(params["maxEpisodes"]):
    count1=0
    for j in range(params["no_envs"]):
        if Actions1[i][j] in np.where(q_star[j]==v_star[j]):
            count1+=1
    optimal1[i]=(count1*100/params["no_envs"])
plt.plot(np.arange(0,params["maxEpisodes"]),optimal1, color='black')
# plt.ylabel('Average Regrets per timesteps')  
# plt.xlabel('Timesteps') 
# plt.title('decayingEpsilonGreedy: % optimal action done vs timestep 2armedBandit') 
# plt.savefig('decayingEpsilonGreedy-2armedBandit  % optimal action done  vs timestep .pdf') 
# plt.close()

Actions1=np.zeros((params["no_envs"],params["maxEpisodes"]))
Q_est1=np.zeros((params["no_envs"],envs[0].action_space.n))
for i in range(params["no_envs"]):
    Q_est,R,actions=softMaxExploration(envs[i],params,decay_type)
    # print(actions)
    # print(np.shape(actions))
    Actions1[i]=actions
    # print(Q_est[-1])
    # Q_est1[i]=Q_est[-1]
Actions1=np.transpose(Actions1)
# print(np.shape(Actions1))
optimal1=np.zeros((params["maxEpisodes"]))     
for i in range(params["maxEpisodes"]):
    count1=0
    for j in range(params["no_envs"]):
        if Actions1[i][j] in np.where(q_star[j]==v_star[j]):
            count1+=1
    optimal1[i]=(count1*100/params["no_envs"])
plt.plot(np.arange(0,params["maxEpisodes"]),optimal1, color='pink')
# plt.ylabel('Average Regrets per timesteps')  
# plt.xlabel('Timesteps') 
# plt.title('softMaxExploration: % optimal action done vs timestep 2armedBandit') 
# plt.savefig(' softMaxExploration-2armedBandit  % optimal action done  vs timestep .pdf') 
# plt.close()

Actions1=np.zeros((params["no_envs"],params["maxEpisodes"]))
Q_est1=np.zeros((params["no_envs"],envs[0].action_space.n))
for i in range(params["no_envs"]):
    Q_est,R,actions=UCBexploration(envs[i],params)
    # print(actions)
    # print(np.shape(actions))
    Actions1[i]=actions
    # print(Q_est[-1])
    # Q_est1[i]=Q_est[-1]
Actions1=np.transpose(Actions1)
# print(np.shape(Actions1))
optimal1=np.zeros((params["maxEpisodes"]))     
for i in range(params["maxEpisodes"]):
    count1=0
    for j in range(params["no_envs"]):
        if Actions1[i][j] in np.where(q_star[j]==v_star[j]):
            count1+=1
    optimal1[i]=(count1*100/params["no_envs"])
plt.plot(np.arange(0,params["maxEpisodes"]),optimal1, color='orange')
# plt.ylabel('Average Regrets per timesteps')  
# plt.xlabel('Timesteps') 
# plt.title('UCBexploration: % optimal action done vs timestep 2armedBandit') 
# plt.savefig(' UCBexploration-2armedBandit  % optimal action done  vs timestep .pdf') 
# plt.close()
plt.ylabel("% Optimal Actions")
plt.xlabel("Steps") 
# plt.legend(["Pure Exploitation","Pure Exploration","Epsilon-Greedy","Decaying Epsilon Greedy","Softmax exploration","UCB Exploration"], loc ="upper left")
plt.title("%Optimal Actions for different Agents: 10 armed")
plt.savefig('%Optimal Actions- 10 armed.pdf') 
